In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [5]:
df = pd.read_csv("data/bank.csv")

In [6]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   int64 
dtypes: int64(8), object(9)
memory usage: 600.6+ KB


In [9]:
df_no_cat, y = df.loc[:, df.dtypes != 'object'].drop('y', axis=1), df['y']

In [17]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(df_no_cat, y, test_size=0.3, random_state=17, stratify=y)

In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [16]:
forest = RandomForestClassifier(random_state=17)

In [21]:
np.mean(cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8495016786335677

In [22]:
forest.fit(df_no_cat_part, y_train_part)

RandomForestClassifier(random_state=17)

In [27]:
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:,1])

0.8631508998911164

In [29]:
label_encoder = LabelEncoder()

In [30]:
df_cat_label_enc = df.copy().drop('y', axis=1)
for col in df.columns[df.dtypes == 'object']:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [31]:
df_cat_label_enc.shape

(4521, 16)

In [32]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(
    df_cat_label_enc, test_size=0.3, stratify=y, random_state=17
)

In [35]:
np.mean(
    cross_val_score(
        forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.8922975749958866

In [36]:
forest.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(random_state=17)

In [37]:

roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.908855334230022